In [3]:
import tangram as tg
import anndata

In [29]:
def slide_to_anndata(slide, label='spot'):
    ad = anndata.AnnData(slide.spot_counts.numpy())
    ad.var_names = slide.genes
    ad.obs_names = [label + '_' + str(i) for i in range(len(slide.spot_counts))]
    return ad

In [14]:
from load_datasets import load_patient
import gene_sets
from cell_regularizer import load_A19_patient_3, load_A3_patient_7, create_cell_reg
from adapt_for_cell_graph_models import adapt_for_cell_graph_models

genes = gene_sets.GENE_SETS['cytassist-autostainer-top1k-by-rank-mean-thresholded']

In [ ]:
datasets = {
    '092146_3': load_patient('092146_3', '__load_as_slide_objects__'),
    '091759_7': load_patient('091759_7', '__load_as_slide_objects__'),
}

In [59]:
scdata = {
    '092146_3': load_A19_patient_3(),
    '091759_7': load_A3_patient_7(),
}

In [60]:
# ad_3_sp = slide_to_anndata(datasets['092146_3'].select_genes(genes), 'spot')
# ad_7_sp = slide_to_anndata(datasets['091759_7'].select_genes(genes), 'spot')

# ad_3_sc = slide_to_anndata(scdata['092146_3'].select_genes(genes), 'cell')
# ad_7_sc = slide_to_anndata(scdata['091759_7'].select_genes(genes), 'cell')

ad_3_sp = slide_to_anndata(datasets['092146_3'], 'spot')
ad_7_sp = slide_to_anndata(datasets['091759_7'], 'spot')

ad_3_sc = slide_to_anndata(scdata['092146_3'], 'cell')
ad_7_sc = slide_to_anndata(scdata['091759_7'], 'cell')

# patient_3_dataset = adapt_for_cell_graph_models(datasets['092146_3'])
# patient_7_dataset = adapt_for_cell_graph_models(datasets['091759_7'])
# patient_3_reg = create_cell_reg([load_A19_patient_3().select_genes(genes)], sample_count=512)
# patient_7_reg = create_cell_reg([load_A3_patient_7().select_genes(genes)], sample_count=512)

tg.pp_adatas(ad_3_sc, ad_3_sp, genes=genes)
tg.pp_adatas(ad_7_sc, ad_7_sp, genes=genes)

ad_3_sp, ad_3_sc

INFO:root:1000 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:17688 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:1000 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:17495 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


(AnnData object with n_obs × n_vars = 6956 × 18072
     obs: 'uniform_density', 'rna_count_based_density'
     var: 'n_cells'
     uns: 'training_genes', 'overlap_genes',
 AnnData object with n_obs × n_vars = 11206 × 17692
     var: 'n_cells'
     uns: 'training_genes', 'overlap_genes')

In [61]:
ad_map_3 = tg.map_cells_to_space(ad_3_sc, ad_3_sp, device='cuda:0')
ad_map_7 = tg.map_cells_to_space(ad_7_sc, ad_7_sp, device='cuda:0')

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 1000 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.758, KL reg: 0.176
Score: 0.938, KL reg: 0.000
Score: 0.948, KL reg: 0.000
Score: 0.952, KL reg: 0.000
Score: 0.953, KL reg: 0.000
Score: 0.955, KL reg: 0.000
Score: 0.955, KL reg: 0.000
Score: 0.956, KL reg: 0.000
Score: 0.956, KL reg: 0.000
Score: 0.956, KL reg: 0.000


INFO:root:Saving results..
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 1000 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.540, KL reg: 0.377
Score: 0.831, KL reg: 0.001
Score: 0.847, KL reg: 0.000
Score: 0.853, KL reg: 0.000
Score: 0.856, KL reg: 0.000
Score: 0.857, KL reg: 0.000
Score: 0.858, KL reg: 0.000
Score: 0.858, KL reg: 0.000
Score: 0.858, KL reg: 0.000
Score: 0.859, KL reg: 0.000


INFO:root:Saving results..


In [44]:
def optimal_matching_loss():
    # Wasserstein loss with cells assigned to the given spot?
    pass

# Axes: [cell i, spot j] -> P(spot j | cell i)
# P(cell i | spot j)P(spot j) = P(spot j | cell i)P(cell i)
# P(cell i | spot j) = P(spot j | cell i)P(cell i)/P(spot j)
# P(spot j) = Sum[P(spot j | cell i)P(cell i)] across all cells i
# ad_map_3.X[0, :].sum() =~= 1
mat_3 = ad_map_3.X
mat_7 = ad_map_7.X

In [62]:
ad_map_3.write("scRNA_tangram_matrix_patient3.h5")
ad_map_7.write("scRNA_tangram_matrix_patient7.h5")

In [56]:
# import numpy as np

# q = 22
# v = np.argpartition(mat_3[:, q], -10)[-10:]
# mat_3[:, q].sum() / mat_3[:, q].sum()

1.0